In [24]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import os
from dotenv import load_dotenv
from numpy import dot
from numpy.linalg import norm

/opt/miniconda3/envs/cricket-rag/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/var/folders/wt/0l5yp46j27z956kvj_jd2gx00000gn/T/ipykernel_10861/384141485.py:4: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [25]:
df = pd.read_csv("data/raw/matches.csv")

In [26]:
print(df.head())


       id   season        city        date match_type player_of_match  \
0  335982  2007/08   Bangalore  2008-04-18     League     BB McCullum   
1  335983  2007/08  Chandigarh  2008-04-19     League      MEK Hussey   
2  335984  2007/08       Delhi  2008-04-19     League     MF Maharoof   
3  335985  2007/08      Mumbai  2008-04-20     League      MV Boucher   
4  335986  2007/08     Kolkata  2008-04-20     League       DJ Hussey   

                                        venue                        team1  \
0                       M Chinnaswamy Stadium  Royal Challengers Bangalore   
1  Punjab Cricket Association Stadium, Mohali              Kings XI Punjab   
2                            Feroz Shah Kotla             Delhi Daredevils   
3                            Wankhede Stadium               Mumbai Indians   
4                                Eden Gardens        Kolkata Knight Riders   

                         team2                  toss_winner toss_decision  \
0        Kolkat

In [27]:
print(f"Shape: {df.shape}")

Shape: (1095, 20)


In [28]:
print(f"Columns: {df.columns.tolist()}")

Columns: ['id', 'season', 'city', 'date', 'match_type', 'player_of_match', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin', 'target_runs', 'target_overs', 'super_over', 'method', 'umpire1', 'umpire2']


In [29]:
df.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan


In [30]:
print(f"Total matches: {len(df)}")
print(f"Seasons covered: {sorted(df['season'].unique())}")

Total matches: 1095
Seasons covered: ['2007/08', '2009', '2009/10', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020/21', '2021', '2022', '2023', '2024']


In [31]:
teams = sorted(df['team1'].unique())
print(f"Total teams: {len(teams)}")
print(teams)

Total teams: 19
['Chennai Super Kings', 'Deccan Chargers', 'Delhi Capitals', 'Delhi Daredevils', 'Gujarat Lions', 'Gujarat Titans', 'Kings XI Punjab', 'Kochi Tuskers Kerala', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Pune Warriors', 'Punjab Kings', 'Rajasthan Royals', 'Rising Pune Supergiant', 'Rising Pune Supergiants', 'Royal Challengers Bangalore', 'Royal Challengers Bengaluru', 'Sunrisers Hyderabad']


In [32]:
df.isnull().sum()

id                    0
season                0
city                 51
date                  0
match_type            0
player_of_match       5
venue                 0
team1                 0
team2                 0
toss_winner           0
toss_decision         0
winner                5
result                0
result_margin        19
target_runs           3
target_overs          3
super_over            0
method             1074
umpire1               0
umpire2               0
dtype: int64

In [33]:
df['winner'].value_counts().head(10)

winner
Mumbai Indians                 144
Chennai Super Kings            138
Kolkata Knight Riders          131
Royal Challengers Bangalore    116
Rajasthan Royals               112
Kings XI Punjab                 88
Sunrisers Hyderabad             88
Delhi Daredevils                67
Delhi Capitals                  48
Deccan Chargers                 29
Name: count, dtype: int64

In [34]:
df['venue'].value_counts().head(5)

venue
Eden Gardens                                 77
Wankhede Stadium                             73
M Chinnaswamy Stadium                        65
Feroz Shah Kotla                             60
Rajiv Gandhi International Stadium, Uppal    49
Name: count, dtype: int64

In [35]:
df.isnull().sum()

id                    0
season                0
city                 51
date                  0
match_type            0
player_of_match       5
venue                 0
team1                 0
team2                 0
toss_winner           0
toss_decision         0
winner                5
result                0
result_margin        19
target_runs           3
target_overs          3
super_over            0
method             1074
umpire1               0
umpire2               0
dtype: int64

In [36]:
# Step 2: Now apply ONLY the correct renames
team_name_fixes = {
    'Delhi Daredevils': 'Delhi Capitals',
    'Royal Challengers Bangalore': 'Royal Challengers Bengaluru',
    'Rising Pune Supergiant': 'Rising Pune Supergiants',
    'Kings XI Punjab': 'Punjab Kings'
}

for col in ['team1', 'team2', 'winner', 'toss_winner']:
    df[col] = df[col].replace(team_name_fixes)

print("✅ Team names standardized!")
print(f"Unique teams now: {df['team1'].nunique()}")
print(sorted(df['team1'].unique()))

✅ Team names standardized!
Unique teams now: 15
['Chennai Super Kings', 'Deccan Chargers', 'Delhi Capitals', 'Gujarat Lions', 'Gujarat Titans', 'Kochi Tuskers Kerala', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Pune Warriors', 'Punjab Kings', 'Rajasthan Royals', 'Rising Pune Supergiants', 'Royal Challengers Bengaluru', 'Sunrisers Hyderabad']


In [37]:
df['city'] = df['city'].fillna('Unknown')
df['winner'] = df['winner'].fillna('No Result')
df['player_of_match'] = df['player_of_match'].fillna('Not Awarded')
df['method'] = df['method'].fillna('Normal')
df['result_margin'] = df['result_margin'].fillna(0)
df['target_runs'] = df['target_runs'].fillna(0)
df['target_overs'] = df['target_overs'].fillna(0)

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

print("✅ All nulls fixed!")
print(df.isnull().sum())

✅ All nulls fixed!
id                 0
season             0
city               0
date               0
match_type         0
player_of_match    0
venue              0
team1              0
team2              0
toss_winner        0
toss_decision      0
winner             0
result             0
result_margin      0
target_runs        0
target_overs       0
super_over         0
method             0
umpire1            0
umpire2            0
year               0
dtype: int64


In [38]:
def match_to_text(row):
    return f"""
IPL Match | Season: {row['season']}
Teams: {row['team1']} vs {row['team2']}
Date: {row['date'].strftime('%d %B %Y')} | Venue: {row['venue']}, {row['city']}
Toss: {row['toss_winner']} won the toss and chose to {row['toss_decision']}
Result: {row['winner']} won by {int(row['result_margin'])} {row['result']}
Player of the Match: {row['player_of_match']}
Method: {row['method']} | Match Type: {row['match_type']}
""".strip()

# Preview 3 matches
for i in [0, 100, 500]:
    print(f"\n--- Match {i} ---")
    print(match_to_text(df.iloc[i]))


--- Match 0 ---
IPL Match | Season: 2007/08
Teams: Royal Challengers Bengaluru vs Kolkata Knight Riders
Date: 18 April 2008 | Venue: M Chinnaswamy Stadium, Bangalore
Toss: Royal Challengers Bengaluru won the toss and chose to field
Result: Kolkata Knight Riders won by 140 runs
Player of the Match: BB McCullum
Method: Normal | Match Type: League

--- Match 100 ---
IPL Match | Season: 2009
Teams: Mumbai Indians vs Rajasthan Royals
Date: 14 May 2009 | Venue: Kingsmead, Durban
Toss: Rajasthan Royals won the toss and chose to bat
Result: Rajasthan Royals won by 2 runs
Player of the Match: SK Warne
Method: Normal | Match Type: League

--- Match 500 ---
IPL Match | Season: 2015
Teams: Kolkata Knight Riders vs Punjab Kings
Date: 09 May 2015 | Venue: Eden Gardens, Kolkata
Toss: Punjab Kings won the toss and chose to bat
Result: Kolkata Knight Riders won by 1 wickets
Player of the Match: AD Russell
Method: Normal | Match Type: League


In [39]:
df['text_chunk'] = df.apply(match_to_text, axis=1)

df.to_csv("data/processed/matches_cleaned.csv", index=False)

print(f"✅ Saved {len(df)} cleaned match records!")
print(f"\nExample text chunk:")
print(df['text_chunk'].iloc[0])

✅ Saved 1095 cleaned match records!

Example text chunk:
IPL Match | Season: 2007/08
Teams: Royal Challengers Bengaluru vs Kolkata Knight Riders
Date: 18 April 2008 | Venue: M Chinnaswamy Stadium, Bangalore
Toss: Royal Challengers Bengaluru won the toss and chose to field
Result: Kolkata Knight Riders won by 140 runs
Player of the Match: BB McCullum
Method: Normal | Match Type: League


In [40]:
print(f"✅ ChromaDB ready: version {chromadb.__version__}")

✅ ChromaDB ready: version 1.5.1


In [41]:
print("Loading embedding model... (first time takes ~1 minute to download)")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded!")

# Test it on one sentence
test = model.encode("Mumbai Indians won the IPL final")
print(f"Embedding shape: {test.shape}")  # Should show (384,)

Loading embedding model... (first time takes ~1 minute to download)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Embedding model loaded!
Embedding shape: (384,)


In [42]:
# This is the core concept of RAG — text becomes numbers
sentence1 = "Mumbai Indians beat Chennai Super Kings"
sentence2 = "MI defeated CSK in the final"
sentence3 = "The price of tomatoes in 2023"

embeddings = model.encode([sentence1, sentence2, sentence3])

# Calculate similarity (closer to 1 = more similar)


def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

sim_12 = cosine_similarity(embeddings[0], embeddings[1])
sim_13 = cosine_similarity(embeddings[0], embeddings[2])

print(f"Similarity (MI vs CSK) ↔ (MI defeated CSK): {sim_12:.3f}")
print(f"Similarity (MI vs CSK) ↔ (tomato prices):   {sim_13:.3f}")
print("\nSee how cricket sentences score much higher? That's how RAG finds relevant data!")

Similarity (MI vs CSK) ↔ (MI defeated CSK): 0.405
Similarity (MI vs CSK) ↔ (tomato prices):   0.097

See how cricket sentences score much higher? That's how RAG finds relevant data!


In [43]:
# Create persistent database (saves to disk)
client = chromadb.PersistentClient(path="data/embeddings")

# Create a collection (like a table in a database)
collection = client.get_or_create_collection(
    name="ipl_matches",
    metadata={"description": "IPL match data 2008-2024"}
)

print(f"✅ ChromaDB collection created!")
print(f"Documents in collection: {collection.count()}")

✅ ChromaDB collection created!
Documents in collection: 1095


In [44]:
# Load cleaned data
import pandas as pd
df = pd.read_csv("data/processed/matches_cleaned.csv")

# Add in batches of 100 (avoids memory issues)
batch_size = 100
total = len(df)

for start in range(0, total, batch_size):
    end = min(start + batch_size, total)
    batch = df.iloc[start:end]
    
    texts = batch['text_chunk'].tolist()
    ids = [f"match_{row['id']}" for _, row in batch.iterrows()]
    
    metadatas = [{
        "team1": row['team1'],
        "team2": row['team2'],
        "winner": row['winner'],
        "season": str(row['season']),
        "venue": row['venue'],
        "player_of_match": row['player_of_match']
    } for _, row in batch.iterrows()]
    
    collection.add(
        documents=texts,
        ids=ids,
        metadatas=metadatas
    )
    
    print(f"✅ Stored matches {start} to {end}")

print(f"\n🏏 Total matches in vector database: {collection.count()}")

✅ Stored matches 0 to 100
✅ Stored matches 100 to 200
✅ Stored matches 200 to 300
✅ Stored matches 300 to 400
✅ Stored matches 400 to 500
✅ Stored matches 500 to 600
✅ Stored matches 600 to 700
✅ Stored matches 700 to 800
✅ Stored matches 800 to 900
✅ Stored matches 900 to 1000
✅ Stored matches 1000 to 1095

🏏 Total matches in vector database: 1095


In [45]:
# Ask a question in plain English
query = "Which matches did Mumbai Indians win at Wankhede Stadium?"

results = collection.query(
    query_texts=[query],
    n_results=3
)

print(f"Query: '{query}'\n")
print("Top 3 most relevant matches found:\n")
for i, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
    print(f"Result {i}:")
    print(doc)
    print(f"Winner: {meta['winner']}")
    print("---")

Query: 'Which matches did Mumbai Indians win at Wankhede Stadium?'

Top 3 most relevant matches found:

Result 1:
IPL Match | Season: 2017
Teams: Mumbai Indians vs Sunrisers Hyderabad
Date: 12 April 2017 | Venue: Wankhede Stadium, Mumbai
Toss: Mumbai Indians won the toss and chose to field
Result: Mumbai Indians won by 4 wickets
Player of the Match: JJ Bumrah
Method: Normal | Match Type: League
Winner: Mumbai Indians
---
Result 2:
IPL Match | Season: 2017
Teams: Mumbai Indians vs Kolkata Knight Riders
Date: 09 April 2017 | Venue: Wankhede Stadium, Mumbai
Toss: Mumbai Indians won the toss and chose to field
Result: Mumbai Indians won by 4 wickets
Player of the Match: N Rana
Method: Normal | Match Type: League
Winner: Mumbai Indians
---
Result 3:
IPL Match | Season: 2016
Teams: Mumbai Indians vs Rising Pune Supergiants
Date: 09 April 2016 | Venue: Wankhede Stadium, Mumbai
Toss: Mumbai Indians won the toss and chose to bat
Result: Rising Pune Supergiants won by 9 wickets
Player of the Mat

In [50]:
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-2.5-flash-lite')

# Quick test
response = model.generate_content("Name 3 IPL teams in one line")
print(response.text)
print("✅ Gemini connected!")

Here are three IPL teams in one line: **Mumbai Indians, Chennai Super Kings, Royal Challengers Bangalore.**
✅ Gemini connected!


In [51]:
def ask_cricket_rag(question, n_results=5):
    
    # Step 1: Search ChromaDB for relevant matches
    results = collection.query(
        query_texts=[question],
        n_results=n_results
    )
    
    # Step 2: Format retrieved matches as context
    context = ""
    for i, doc in enumerate(results['documents'][0], 1):
        context += f"Match {i}:\n{doc}\n\n"
    
    # Step 3: Build prompt for Gemini
    prompt = f"""You are an expert IPL cricket analyst. 
Use ONLY the match data below to answer the question accurately.
If the answer isn't in the data, say so honestly.

MATCH DATA:
{context}

QUESTION: {question}

Give a clear, detailed answer with specific stats and match details:"""
    
    # Step 4: Get Gemini's answer
    response = model.generate_content(prompt)
    
    return response.text, results['documents'][0]

print("✅ RAG function ready!")

✅ RAG function ready!


In [52]:
questions = [
    "How did Mumbai Indians perform at Wankhede Stadium?",
    "Which player won the most Player of the Match awards?",
    "What was the biggest win margin in IPL history?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"❓ {q}")
    print('='*50)
    answer, _ = ask_cricket_rag(q)
    print(answer)


❓ How did Mumbai Indians perform at Wankhede Stadium?
Based on the provided match data, here's how Mumbai Indians performed at Wankhede Stadium:

**Overall Record at Wankhede Stadium:**

*   **Matches Played:** 5
*   **Wins:** 4
*   **Losses:** 1

**Match-by-Match Breakdown:**

*   **Match 1 (2016):** Mumbai Indians vs Rising Pune Supergiants. Mumbai Indians chose to bat after winning the toss. They **lost** this match by 9 wickets.
*   **Match 2 (2007/08):** Mumbai Indians vs Chennai Super Kings. Mumbai Indians chose to field after winning the toss. They **won** this match by 9 wickets.
*   **Match 3 (2017):** Mumbai Indians vs Sunrisers Hyderabad. Mumbai Indians chose to field after winning the toss. They **won** this match by 4 wickets.
*   **Match 4 (2017):** Mumbai Indians vs Gujarat Lions. Mumbai Indians chose to field after winning the toss. They **won** this match by 6 wickets.
*   **Match 5 (2013):** Mumbai Indians vs Delhi Capitals. Mumbai Indians chose to bat after winning 

In [53]:
def cricket_analytics(question):
    q = question.lower()
    
    # Most POTM awards
    if "player of the match" in q or "potm" in q or "most awards" in q:
        potm = df[df['player_of_match'] != 'Not Awarded']['player_of_match'].value_counts().head(5)
        return f"🏆 Most Player of the Match Awards:\n{potm.to_string()}"
    
    # Biggest win margin
    if "biggest win" in q or "largest margin" in q or "biggest margin" in q:
        biggest = df[df['result'] == 'runs'].nlargest(3, 'result_margin')[
            ['team1','team2','winner','result_margin','season','venue']
        ]
        return f"💥 Biggest Wins by Runs:\n{biggest.to_string(index=False)}"
    
    # Most wins by team
    if "most wins" in q or "most successful" in q or "best team" in q:
        wins = df[df['winner'] != 'No Result']['winner'].value_counts().head(5)
        return f"🏏 Most Wins in IPL History:\n{wins.to_string()}"
    
    # Head to head
    if "vs" in q or "head to head" in q or "against" in q:
        teams = [t for t in df['team1'].unique() if t.lower() in q]
        if len(teams) >= 2:
            h2h = df[
                ((df['team1'] == teams[0]) & (df['team2'] == teams[1])) |
                ((df['team1'] == teams[1]) & (df['team2'] == teams[0]))
            ]
            wins = h2h['winner'].value_counts()
            return f"⚔️ Head to Head ({teams[0]} vs {teams[1]}):\n{wins.to_string()}\nTotal matches: {len(h2h)}"
    
    return None  # Fall back to RAG


def smart_cricket_answer(question, n_results=5):
    # First try analytics layer
    analytics_answer = cricket_analytics(question)
    if analytics_answer:
        print("📊 [Answered by Analytics Layer]")
        return analytics_answer
    
    # Otherwise use RAG
    print("🤖 [Answered by RAG + Gemini]")
    answer, _ = ask_cricket_rag(question, n_results)
    return answer

print("✅ Smart answer function ready!")

✅ Smart answer function ready!


In [54]:
questions = [
    "How did Mumbai Indians perform at Wankhede Stadium?",
    "Which player won the most Player of the Match awards?",
    "What was the biggest win margin in IPL history?",
    "Mumbai Indians vs Chennai Super Kings head to head",
    "Most successful team in IPL history"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"❓ {q}")
    print('='*50)
    print(smart_cricket_answer(q))


❓ How did Mumbai Indians perform at Wankhede Stadium?
🤖 [Answered by RAG + Gemini]
Here's how the Mumbai Indians performed at Wankhede Stadium based on the provided data:

*   **Match 1 (2016 vs. Rising Pune Supergiants):** Mumbai Indians won the toss and chose to bat, but **lost by 9 wickets**.
*   **Match 2 (2008 vs. Chennai Super Kings):** Mumbai Indians won the toss and chose to field, and **won by 9 wickets**.
*   **Match 3 (2017 vs. Sunrisers Hyderabad):** Mumbai Indians won the toss and chose to field, and **won by 4 wickets**.
*   **Match 4 (2017 vs. Gujarat Lions):** Mumbai Indians won the toss and chose to field, and **won by 6 wickets**.
*   **Match 5 (2013 vs. Delhi Capitals):** Mumbai Indians won the toss and chose to bat, and **won by 44 runs**.

In summary, out of the 5 matches played at Wankhede Stadium for which data is available, Mumbai Indians have won **4 matches** and lost **1 match**.

❓ Which player won the most Player of the Match awards?
📊 [Answered by Analyti